# Satellite-based monitoring of dry and wet conditions using Standardized Precipitation Index (SPI)

Benny Istanto | [benny.istanto@wfp.org](benny.istanto@wfp.org) | Earth Observation and Climate Analyst | United Nations World Food Programme, Indonesia

------------

The SPI analysis is following the training conducted in 28 Jan 2020 by [NASA ARSET](https://arset.gsfc.nasa.gov) on **Application of GPM IMERG Reanalysis for Assessing Extreme Dry and Wet Periods**. Link: https://arset.gsfc.nasa.gov/water/webinars/IMERG-2020

Some of the step have been modified and adjusted based on experiencing of several problems during the training, and latest version of [Climate Indices in Python](https://github.com/monocongo/climate_indices) software is used in this tutorial. While NASA ARSET training still used the official release version from [U.S. Drought Portal](https://www.drought.gov/drought/python-climate-indices)

This step-by-step guide was tested using macOS Catalina 10.15


## 1. Objectives
- Learn how to bulk download IMERG data from NASA GES DISC
- Determine how to calculate the Standardized Precipitation Index (SPI) for assessing extreme dry and wet periods

### 1.1. Outline
- Background on the Standardized Precipitation Index (SPI)
    - SPI Interpretation
- Software and Data Preparation
    - Case Study: Bali (Indonesia)
    - Install Supporting Software
    - Data Acquisition
    - Configure the Python environment
    - Pre-process data using NetCDF Operator ([NCO](http://nco.sourceforge.net/nco.html))
- Running SPI code    
    - Pre-compute SPI distribution fitting variables (Optional analysis)
- Interpret the result using Panoply

---

## 2. Background on the Standardized Precipitation Index
First developed by T.B. McKee et al. (1993) and used by Guttman (1999)
- Used for estimating meteorological conditions based on precipitation alone
- Wet or dry conditions can be monitored on a variety of time scales from sub seasonal to interannual
- Can be compared across regions with markedly difference climates
- Does not consider the intensity of precipitation and its potential impacts on runoff, streamflow, and water availability
- Expressed as the number of standard deviations from the long term mean, for a normally distributed random variable, and fitted probability distribution for the actual precipitation record
- SPI values < 1 indicate a condition of drought, the more negative the value the more severe the drought condition. SPI values > +1 indicate wetter conditions compared to a climatology

### 2.1. SPI Interpretation
https://drought.unl.edu/droughtmonitoring/SPI.aspx

- 1 month: Similar to a map displaying the percent of normal precipitation for a month. Reflects relatively short term conditions. Its application can be related closely with short term soil moisture and crop stress.
- 3 month: Provides a comparison of the precipitation over a specific 3 month period with the precipitation totals from the same 3 month period for all the years included in the historical record. Reflects short and medium term moisture conditions and provides a seasonal estimation of precipitation.
- 6 month: Compares the precipitation for that period with the same 6 month period over the historical record. A 6 month SPI can be very effective in showing the precipitation over distinct seasons and may be associated with anomalous streamflow and reservoir levels.
- 9 month: Provides an indication of precipitation patterns over a medium time scale. SPI values below 1.5 for these time scales are usually a good indication that significant impacts are occurring in agriculture and may be showing up in other sectors as well.
- 12 month: Reflects long term precipitation patterns. Longer SPIs tend toward zero unless a specific trend is taking place. SPIs of these time scales are probably tied to streamflow, reservoir levels, and even groundwater levels at the longer time scales. In some locations of the country, the 12 month SPI is most closely related with the Palmer Index, and the two indices should reflect similar conditions.

![SPI Interpretation](Exercise/img/normalcurve.png)

SPI labels and their relationship to the normal curve. The intensity implied by each label corresponds to the degree of removal from mean conditions (i.e., SPI=0). The percentages printed within the regions bounded by the dashed lines indicate the probability for SPI values to fall within that region only. (Source: J. Keyantash)

### 2.2. Reference
- Guttman, N. B., 1999: Accepting the Standardized Precipitation Index: A calculation algorithm. J. Amer. Water Resour. Assoc., 35(2), 311-322. [Link](https://climatedataguide.ucar.edu/climate-data/standardized-precipitation-index-spi)
- Keyantash, John & National Center for Atmospheric Research Staff (Eds). "The Climate Data Guide: Standardized Precipitation Index (SPI)." Retrieved from https://climatedataguide.ucar.edu/climate-data/standardized-precipitation-index-spi
- Lloyd Hughes, B., and M. A. Saunders, 2002: A drought climatology for Europe. Int. J. Climatol., DOI:10.1002/joc.846 [Link](https://rmets.onlinelibrary.wiley.com/doi/epdf/10.1002/joc.846)
- McKee, T.B., N. J. Doesken, and J. Kliest, 1993: The relationship of drought frequency and duration to time scales. In Proceedings of the 8th Conference of Applied Climatology, 17 22 January, Anaheim, CA. American Meterological Society, Boston, MA. 179-18. [Link](https://www.droughtmanagement.info/literature/AMS_Relationship_Drought_Frequency_Duration_Time_Scales_1993.pdf)
- National Drought Mitigation Center (NDMC) at the University of Nebraska Lincoln. [Link](https://drought.unl.edu/droughtmonitoring/SPI.aspx)
- World Meteorological Organization (WMO), 2012: Standardized Precipitation Index User Guide. [Link](https://library.wmo.int/doc_num.php?explnum_id=7768)
- Climate Indices in Python https://climate-indices.readthedocs.io/en/latest/

---

## 3. Software and Data Preparation

Case study: Bali island, Indonesia. **Why Bali? Who doesn't know Bali?**

### 3.1. Install Supporting Software
**If you encounter a problem, please look for a online solution.** The installation and configuration described below is performed using a bash shell, either on Linux, Windows, or macOS.
Windows users will need to install and configure a bash shell in order to follow the usage shown below. We recommended either [babun](https://babun.github.io/) or [Cygwin](https://www.cygwin.com/) or [Windows Subsytem for Linux](https://docs.microsoft.com/en-us/windows/wsl/install-win10) for this purpose.
1. If you are new to using Bash refer to the following lessons with Software Carpentry: http://swcarpentry.github.io/shell-novice/
2. If you don't have [Homebrew](https://brew.sh), you can install it by pasting below code in your macOS terminal:
    - Install ```Homebrew```: ```/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install.sh)"```
3. Install wget (for downloading data) and rename (for renaming data). Use Hombrew to install it by pasting below code in your macOS terminal
    - wget ```brew install wget```
    - rename ```brew install rename```
4. Download and install [Panoply Data Viewer](https://www.giss.nasa.gov/tools/panoply/) from [NASA GISS](https://www.giss.nasa.gov/tools/panoply/download/) on your machine:
    - macOS: https://www.giss.nasa.gov/tools/panoply/download/PanoplyMacOS-4.11.4.dmg
    - Windows: https://www.giss.nasa.gov/tools/panoply/download/PanoplyWin-4.11.4.zip
    - Linux: https://www.giss.nasa.gov/tools/panoply/download/PanoplyJ-4.11.4.zip

### 3.2. Data Acquisition
1. Download monthly IMERG data from GES DISC:
    - Using a web browser, go to NASA Goddard Earth Sciences (GES) Data and Information Services Center (DISC): https://disc.gsfc.nasa.gov/
    - Type “IMERG” in the search bar and click on the search icon
    - Select IMERG Version 6 Level 3 data at “monthly” temporal resolution and click on the “Subset/Get Data” icon
    
    ![IMERG monthly](Exercise/img/imerg-data.png)
    
    - Leave the default date range since we want the entire time series
    - Under Spatial Subset enter ```114.3, -9, 115.8, -8``` This spatial subset is for **Bali island**
    - Under Variables select only ```precipitation```
    - Leave the default parameters under Grid
    - Under File Format select "netCDF"
    - Click Get Data
    
    ![Download options](Exercise/img/download-options.png)
    
    - Data links windows will popup and you may click **"Download links list"**
    
    ![Data Links](Exercise/img/data-links.png)
    
    - You will get a txt file with similar filename like this one **subset_GPM_3IMERGM_06_20200703_065511.txt**
    - Move this file into your working directory (this case we have folder ```/Exercise/IMERG_originalfiles``` to save the txt file)
    - Navigate your terminal to folder ```/Exercise/IMERG_originalfiles``` and type this code to download the data ```wget -c -i subset_GPM_3IMERGM_06_20200703_065511.txt```
    
    ![Wget](Exercise/img/wget.png)
    
    - If you are lazy to follow the process of downloading data, for convenience these data are made available on via this link: https://github.com/wfpidn/SPI/blob/master/Data/IMERG_originalfiles.zip
    - Once downloaded, unzip ```IMERG_originalfiles```
2. Rename all the data into friendly filename
    - If you check the data in folder ```IMERG_originalfiles```, you will find the data with filename something like
    ```HTTP_services.cgi?FILENAME=%2Fdata%2FGPM_L3%2FGPM_3IMERGM.06%2F2020%2F3B-MO.MS.MRG.3IMERG.20200301-S000000-E235959.03.V06B.HDF5&FORMAT=bmM0Lw&BBOX=-9,114.3,-8,115.8&LABEL=3B-MO.MS.MRG.3IMERG.20200301-S000000-E235959.03.V06B.HDF5.SUB.nc4```
    
    ![Original Filenames](Exercise/img/orig-filenames.png)
    
    - We need to rename it all the file into friendly filename like this
    ```3B-MO.MS.MRG.3IMERG.20200301-S000000-E235959.03.V06B.HDF5.SUB.nc4```
    - If you follow the download process, you may create a duplicate folder and it's contents for ```IMERG_originalfiles``` and rename it to ```IMERG_mmhr``` (just in case something happen to you downloaded files). But if you not follow the download process but downloaded ```IMERG_originalfiles.zip``` folder, you are good.
    - We will use [regular expression](https://en.wikipedia.org/wiki/Regular_expression) and remove the first 117 characters in the filename. Using ```rename``` command, make sure you are in newly duplicate folder ```IMERG_mmhr``` directory in your terminal, type this code:
    ```rename 's/.{171}//g' *.nc4```
    
    ![Rename](Exercise/img/rename.png) 
    
    ![Renamed Filenames](Exercise/img/rename-filenames.png) 


### 3.3. Configure the Python environment
This code for calculating SPI is written in Python 3. It is recommended to use either the [Miniconda3](https://docs.conda.io/en/latest/miniconda.html) (minimal Anaconda) or Anaconda3 distribution. The below instructions will be Anaconda specific (although relevant to any Python [virtual environment](https://virtualenv.pypa.io/en/stable/)), and assume the use of a bash shell.

A new Anaconda [environment](https://conda.io/docs/using/envs.html) can be created using the [conda](https://conda.io/docs/) environment management system that comes packaged with Anaconda. In the following examples, we’ll use an environment named climate_indices (any environment name can be used instead of climate_indices) which will be created and populated with all required dependencies through the use of the provided ```setup.py``` file.

##### Python
1. Download and install Anaconda Python version 3.7 on your machine:
    - macOS: https://repo.anaconda.com/archive/Anaconda3-2020.02-MacOSX-x86_64.pkg
    - Windows: https://repo.anaconda.com/archive/Anaconda3-2020.02-Windows-x86_64.exe
    - Linux: https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh
2. Or you can use Miniconda
    - macOS: https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-x86_64.pkg
    - Windows: https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe
    - Linux: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
3. For **Windows users**, follow [this](https://arset.gsfc.nasa.gov/sites/default/files/water/20-IMERG/IMERG_Week2_FINAL.pdf) procedure, starting from pages 16
4. First, create the Python environment:


In [ ]:
conda create -n climate_indices python=3.7

5. The environment created can now be ‘activated’:

In [ ]:
source activate climate_indices

6. Install ```climate-indices``` package. Once the environment has been activated then subsequent Python commands will run in this environment where the package dependencies for this project are present. Now the package can be added to the environment along with all required modules (dependencies) via [pip](https://pip.pypa.io/en/stable/):

In [ ]:
pip install climate-indices

#### NCO

7. NetCDF Operators (NCO) is a requirement and must be installed for utilization of this package. Instructions for installation on various platforms is available [here](http://nco.sourceforge.net/#Executables//) and other information related to NCO command can be found at the link: http://nco.sourceforge.net/nco.html. If using an Anaconda environment as advised above then it’s as simple as running the following command within the activated conda environment:

In [ ]:
conda install -c conda-forge nco

### 3.4. Preprocess data using NCO
1. To read header contents of a netCDF file: ```ncdump h filename.nc4```. Navigate to folder IMERG_mmmonth, type and execute below code:

In [ ]:
ncdump -h 3B-MO.MS.MRG.3IMERG.20000601-S000000-E235959.06.V06B.HDF5.SUB.nc4

You will get information (dimension, variables, global attribute) about the data

![NCDUMP](Exercise/img/ncdump.png) 

2. The original downloaded files from GPM IMERG is in mm/hr, while to calculate monthly SPI the data must be in mm/month, we need to do a conversion process using ```ncap2``` (arithmetic operator for NetCDF files) command by multiplying number of day in month for each data with 24hour. 

3. Arrange IMERG data files based on number of days within a month, dont forget to create a folder to store the files:
    - ```/Exercise/IMERG_mmhr/Day28```
    - ```/Exercise/IMERG_mmhr/Day29```
    - ```/Exercise/IMERG_mmhr/Day30```
    - ```/Exercise/IMERG_mmhr/Day31```, and new folder 
    - ```/Exercise/IMERG_mm_month``` to save the converted files.

>    - Navigate to ```Day28``` folder in terminal, type this code:

In [ ]:
for fl in *.nc4; do ncap2 -s 'precipitation=672*precipitation' $fl ../../IMERG_mm_month/$fl; done

>    - Navigate to ```Day29``` folder in terminal, type this code:

In [ ]:
for fl in *.nc4; do ncap2 -s 'precipitation=696*precipitation' $fl ../../IMERG_mm_month/$fl; done

>    - Navigate to ```Day30``` folder in terminal, type this code:

In [ ]:
for fl in *.nc4; do ncap2 -s 'precipitation=720*precipitation' $fl ../../IMERG_mm_month/$fl; done

>    - Navigate to ```Day31``` folder in terminal, type this code:

In [ ]:
for fl in *.nc4; do ncap2 -s 'precipitation=744*precipitation' $fl ../../IMERG_mm_month/$fl; done

4. Navigate to ```IMERG_mm_month``` folder in terminal. Loops all files in the folder ```IMERG_mm_month``` to make ```time``` the record dimension/variable used for concatenating using ```Ncks``` command

In [ ]:
for fl in *.nc4; do ncks O mk_rec_dmn time $fl $fl; done

5. Concatenates all ```.nc4``` files in ```IMERG_mm_month``` folder into one ```.nc4``` file named ```IMERG_concat.nc4``` using ```ncrcat``` command

In [ ]:
ncrcat -h *.nc4 IMERG_concat.nc4

6. Check the header

In [ ]:
ncdump -h IMERG_concat.nc4

7. And the variables for ```precipitation``` is ```time,lon,lat``` but SPI calculation required:
    - ```lat,lon,time``` or
    - ```time,lat,lon```
    
    ![NCDUMP IMERG concat](Exercise/img/ncdump_concat.png)
   

8. Let's re-order the variables into ```time,lat,lon``` using ```ncpdq``` command, to be able running the SPI code in Python

In [ ]:
ncpdq -a time,lat,lon IMERG_concat.nc4 IMERG_concat_ncpdq0.nc4

9. Check again the header for ```IMERG_concat_ncpdq0.nc4```

In [ ]:
ncdump -h IMERG_concat_ncpdq0.nc4

10. And the variables for ```precipitation``` is ```time,lat,lon```, it means the result is correct. But the unit still in ```mm/hr```, we need to edit to ```mm```
    
    ![NCDUMP IMERG concat ncpdq0](Exercise/img/ncdump_ncpdq0.png)
   

>#### **Notes on re-ordering process** (Case by case)
    
>    - After re-ordering the variables, sometimes user experience ```lat``` or ```lon``` dimension becomes ```UNLIMITED``` which is wrong. The ```time``` dimension should be the ```UNLIMITED``` dimension.
>
>    ![NCDUMP IMERG lat unlimited](Exercise/img/lat_unlimited.png)
>
>    - Fortunately you can do this to fix the ```lat``` or ```lon``` dimension who becomes ```UNLIMITED``` using ```ncks``` command below:

In [ ]:
ncks --fix_rec_dmn lat IMERG_concat_ncpdq0.nc4.nc4 -o outfixed.nc4 ; mv outfixed.nc4 IMERG_concat_ncpdq0.nc4

>    - And to make ```UNLIMITED``` the ```time``` dimension again using ```ncks``` command below:

In [ ]:
ncks --fix_rec_dmn time IMERG_concat_ncpdq0.nc4 -o outfunlim.nc4 ; mv outunlim.nc4 IMERG_concat_ncpdq0.nc4

>    - If you don't come accross the problem, ```lat``` or ```lon``` dimension becomes ```UNLIMITED```, then skip above process and go directly to step 11.

11. SPI code does not recognized unit ```mm/hr``` or ```mm/month```, we need to edit into ```mm```. To edit the unit attribute names, we will use ```ncatted``` command, follow below code:

In [ ]:
ncatted -a units,precipitation,modify,c,'mm' IMERG_concat_ncpdq0.nc4 IMERG_concat_ncpdq1.nc4

12. Check again the header for IMERG_concat_ncpdq1.nc4, to make sure everything is correct.

In [ ]:
ncdump -h IMERG_concat_ncpdq1.nc4

13. And the ```units``` already in ```mm```
    
    ![NCDUMP IMERG concat ncpdq1](Exercise/img/ncdump_ncpdq1.png)
   

14. Once this has completed the dataset can be used as input to this package for computing SPI. From above picture, some of the precipitation attribute are still wrong: ```DimensionNames``` and ```Units```. We can leave it as is, SPI code will only read ```units``` and ```variables``` ```precipitation(time,lat,lon)```

---

## 4. Running SPI code

1. We will use ```IMERG_concat_ncpdq1.nc4``` as ```input```, let's move this data to ```Input``` folder.
    - Move using ```mv oldlocation newlocation``` command: ```mv /IMERG_mm_month/IMERG_concat_ncpdq1.nc4 /Input/IMERG_concat_ncpdq1.nc4```

2. Using the Bash shell from your macOS terminal, run the following code:

In [ ]:
process_climate_indices --index spi --periodicity monthly --netcdf_precip /Input/IMERG_concat_ncpdq1.nc4 --var_name_precip precipitation --output_file_base /Output/IMERG --scales 1 2 3 6 9 12 24 36 48 60 72 --calibration_start_year 2000 --calibration_end_year 2020 --multiprocessing all

3. The above command will compute SPI (standardized precipitation index, both gamma and Pearson Type III distributions) from an input precipitation dataset (in this case, the IMERG precipitation dataset provided in the example inputs directory). The input dataset is monthly data and the calibration period used will be Jun 2000 through Mar 2020. The index will be computed at 1 to 72-month timescales. The output files will be:
    - 1-month: ```/Output/IMERG_spi_gamma_01.nc```
    - 2-month: ```/Output/IMERG_spi_gamma_02.nc```
    - 3-month: ```/Output/IMERG_spi_gamma_03.nc```
    - 6-month: ```/Output/IMERG_spi_gamma_06.nc```
    - 9-month: ```/Output/IMERG_spi_gamma_09.nc```
    - 12-month: ```/Output/IMERG_spi_gamma_12.nc```
    - 24-month: ```/Output/IMERG_spi_gamma_24.nc```
    - 36-month: ```/Output/IMERG_spi_gamma_36.nc```
    - 48-month: ```/Output/IMERG_spi_gamma_48.nc```
    - 60-month: ```/Output/IMERG_spi_gamma_60.nc```
    - 72-month: ```/Output/IMERG_spi_gamma_72.nc```
    - 1-month: ```/Output/IMERG_spi_pearson_01.nc```
    - 2-month: ```/Output/IMERG_spi_pearson_02.nc```
    - 3-month: ```/Output/IMERG_spi_pearson_03.nc```
    - 6-month: ```/Output/IMERG_spi_pearson_06.nc```
    - 9-month: ```/Output/IMERG_spi_pearson_09.nc```
    - 12-month: ```/Output/IMERG_spi_pearson_12.nc```
    - 24-month: ```/Output/IMERG_spi_pearson_24.nc```
    - 36-month: ```/Output/IMERG_spi_pearson_36.nc```
    - 48-month: ```/Output/IMERG_spi_pearson_48.nc```
    - 60-month: ```/Output/IMERG_spi_pearson_60.nc```
    - 72-month: ```/Output/IMERG_spi_pearson_72.nc```

Parallelization will occur utilizing all CPUs.

### 4.1. Pre-compute SPI distribution fitting variables (Optional analysis)

1. In order to pre-compute fititng parameters for later use as inputs to subsequent SPI calculations we can save both gamma and Pearson distributinon fitting parameters to NetCDF, and later use this file as input for SPI calculations over the same calibration period.

In [ ]:
spi --periodicity monthly --scales 1 2 3 6 9 12 24 36 48 60 72 --calibration_start_year 2000 --calibration_end_year 2020 --netcdf_precip /Input/IMERG_concat_ncpdq1.nc4 --var_name_precip precipitation --output_file_base /Output/IMERG --multiprocessing all --save_params /Input/IMERG_fitting.nc --overwrite

In [ ]:
spi --periodicity monthly --scales 1 2 3 6 9 12 24 36 48 60 72 --calibration_start_year 2000 --calibration_end_year 2020 --netcdf_precip /Input/IMERG_concat_ncpdq1.nc4 --var_name_precip precipitation --output_file_base /Output/IMERG --multiprocessing all --load_params /Input/IMERG_fitting.nc

2. In the above example we demonstrate how distribution fitting parameters can be saved as NetCDF. This fittings NetCDF can then be used as pre-computed variables in subsequent SPI computations. Inital command computes both distribution fitting values and SPI for various month scales. The distribution fitting variables are written to the file specified by the ```–save_params``` option. The second command also computes SPI but instead of computing the distribution fitting values it loads the pre-computed fitting values from the NetCDF file specified by the ```–load_params``` option.

    ![SPI](Exercise/img/spi.png)

---

## 5. Interpret the result using Panoply

Let see the result

1. Launch the Panoply desktop application
2. Open the SPI file ```/Output/IMERG_spi_gamma_01.nc``` in Panoply

![Panoply Data](Exercise/img/panoply-datasets.png)

3. From the Datasets tab select spi_gamma_03 and click Create Plot

![Panoply Plot](Exercise/img/panoply-createplot.png)

4. In the Create Plot window select ‘Create a georeferenced <<Longitude Latitude>> plot’ and click Create
5. When the Plot window opens:
    - Array tab: Change the time into 238 to view the latest/last data ~ Mar 2020
    - Scale tab: Change value on Min -3, Max 3, Major 6, Color Table CB_RdBu_09.cpt
    - Map tab: Change value on Center on Lon 115.0, Lat -8.5, then Zoom in the map through menu-editor Plot > Zoom Plot In few times until Bali island bigger.
    - Overlays tab: Change Overlay 1 to MWDB_Coasts_Countries_1.cnob
    - To explore the values in the array, click on the Array 1 tab at the top of the window
    - SPI1 - Mar 2020
    
    ![Panoply SPI](Exercise/img/panoply-spi-mar2020.png)
    
    - SPI1 - Feb 2020
    
    ![Panoply SPI](Exercise/img/panoply-spi-feb2020.png)
    
    - SPI1 - Jan 2020
    
    ![Panoply SPI](Exercise/img/panoply-spi-jan2020.png)

## End